In [1]:
import json
import pandas as pd
import os

In [ ]:
data_path = "./"
embed_path = "./output/train/embed"

In [9]:
import pickle

def load_dataset(file_path, prefix_path=data_path):
    ds = None
    full_path = os.path.join(prefix_path, file_path)
    if os.path.splitext(file_path)[-1] == ".csv":
        ds = pd.read_csv(full_path)
    elif os.path.splitext(file_path)[-1] == ".jsonl":
        ds = pd.read_json(full_path, lines=True)
    elif os.path.splitext(file_path)[-1] == ".pickle":
        with open(full_path, "rb") as f:
            ds = pickle.load(f)
    return ds

# Words-embedding

## Load training datasets

In [10]:
real_data_file = 'webtext.train.jsonl.clean100k.csv'
fake_data_file = 'xl-1542M.train.jsonl.clean100k.csv'

In [11]:
df_real = load_dataset(real_data_file)
df_fake = load_dataset(fake_data_file)

corpus = df_real["text"].to_list() + df_fake["text"].to_list()
labels = [0 for _ in range(len(df_real))] + [1 for _ in range(len(df_fake))]

## Build vocabulary

In [12]:
from nlp_engine.preprocessing import transformers as tfs
from sklearn.pipeline import make_pipeline

In [13]:
tokenized_corpus = make_pipeline(
    tfs.WordTokenizer(), 
    tfs.WordsFilter(drop_symbols=False, drop_digits=True)
).fit_transform(corpus)

In [14]:
from nlp_engine.analysis import vocabulary

In [15]:
vocab_real = vocabulary.get_vocabulary(tokenized_corpus[:len(df_real)])
vocab_fake = vocabulary.get_vocabulary(tokenized_corpus[len(df_real):])

In [16]:
vocab_shared = vocab_real.intersection(vocab_fake)

In [17]:
print(f'Size of "real" vocabulary: {len(vocab_real)}\nSize of "fake" vocabulary: {len(vocab_fake)}\nSize of intersection: {len(vocab_shared)}')

Size of "real" vocabulary: 290187
Size of "fake" vocabulary: 511432
Size of intersection: 160741


## Compute TF-IDF embedding

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [19]:
def identity(x):
    return x

In [20]:
words_pipeline = make_pipeline(
    tfs.WordTokenizer(),
    tfs.WordsFilter(
        drop_symbols=False, 
        drop_digits=True, 
        whitelist=vocab_shared),
    TfidfVectorizer(
        ngram_range=(1,3), 
        max_features=1000000, 
        sublinear_tf=True, 
        tokenizer=identity, 
        preprocessor=identity),
    TruncatedSVD(n_components=500)
)

In [21]:
%%time
words_pipeline.fit(corpus, labels)

/Users/alessandro/Documents/dev/ml-env/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CPU times: user 34min 47s, sys: 21min 20s, total: 56min 8s
Wall time: 47min 46s


Pipeline(steps=[('wordtokenizer', WordTokenizer()),
                ('wordsfilter',
                 WordsFilter(drop_digits=True,
                             whitelist={'!', '"', '#', '$', '%', '&', "'", '(',
                                        ')', '*', '+', ',', '-', '.', '/', ':',
                                        ';', '<', '=', '>', '?', '@', '[', '\\',
                                        ']', '^', '`', 'a', 'a0', 'a02', ...})),
                ('tfidfvectorizer',
                 TfidfVectorizer(max_features=1000000, ngram_range=(1, 3),
                                 preprocessor=<function identity at 0x2de2ca160>,
                                 sublinear_tf=True,
                                 tokenizer=<function identity at 0x2de2ca160>)),
                ('truncatedsvd', TruncatedSVD(n_components=500))])

Transform training data and store it

In [23]:
real_embed = words_pipeline.transform(df_real["text"].to_list())
fake_embed = words_pipeline.transform(df_fake["text"].to_list())

In [24]:
import pickle

embed_path = "./output/train/embed"

with open(os.path.join(
    embed_path, f"{real_data_file.replace('.jsonl.clean100k.csv','')}.human_embed.pickle"), "wb") as f:
    pickle.dump(real_embed, f)
with open(os.path.join(
    embed_path, f"{fake_data_file.replace('.jsonl.clean100k.csv','')}.machine_embed.pickle"), "wb") as f:
    pickle.dump(fake_embed, f)

## Embed test datasets

In [27]:
test_path = "./data/test"
test_dataset_files = sorted([f for f in os.listdir(test_path) if os.path.isfile(os.path.join(test_path, f)) and f[0]!="."])

In [29]:
embed_path = "./output/test/embed"

for test in test_dataset_files:
    with open(os.path.join(test_path, test), "rb") as f:
        X = pickle.load(f)
    X_embed = words_pipeline.transform(X)
    with open(os.path.join(embed_path,f"{os.path.splitext(test)[0]}_embed.pickle"), "wb") as f:
        pickle.dump(X_embed, f)

# TAG-embedding

In [30]:
data_path = "./data/tag"
embed_path = "./data/tag/embed"

## Load training datasets

In [31]:
real_data_file = 'webtext.train.jsonl.clean100k.csv.tag.pickle'
fake_data_file = 'xl-1542M.train.jsonl.clean100k.csv.tag.pickle'

In [32]:
ds_real = load_dataset(real_data_file, data_path)
ds_fake = load_dataset(fake_data_file, data_path)

In [34]:
corpus = ds_real + ds_fake
labels = [0 for _ in range(len(ds_real))] + [1 for _ in range(len(ds_fake))]

## Compute TF-IDF embedding

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

In [36]:
def identity(x):
    return x

In [37]:
tags_pipeline = make_pipeline(
    TfidfVectorizer(
        ngram_range=(3,5), 
        max_features=1000000, 
        sublinear_tf=True, 
        tokenizer=identity, 
        preprocessor=identity),
    TruncatedSVD(n_components=300)
)

In [38]:
%%time
tags_pipeline.fit(corpus, labels)

CPU times: user 26min 42s, sys: 2min 19s, total: 29min 1s
Wall time: 21min 9s


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_features=1000000, ngram_range=(3, 5),
                                 preprocessor=<function identity at 0x5388f8160>,
                                 sublinear_tf=True,
                                 tokenizer=<function identity at 0x5388f8160>)),
                ('truncatedsvd', TruncatedSVD(n_components=300))])

Transform training data and store it

In [40]:
real_embed = tags_pipeline.transform(ds_real)
fake_embed = tags_pipeline.transform(ds_fake)

In [41]:
import pickle

embed_path = "./output/train/embed"

with open(os.path.join(
    embed_path, f"{real_data_file.replace('.jsonl.clean100k.csv','').replace('.pickle','')}.human_embed.pickle"), "wb") as f:
    pickle.dump(real_embed, f)
with open(os.path.join(
    embed_path, f"{fake_data_file.replace('.jsonl.clean100k.csv','').replace('.pickle','')}.machine_embed.pickle"), "wb") as f:
    pickle.dump(fake_embed, f)

## Embed test datasets

In [42]:
test_path = "./data/tag/test"
embed_path = "./output/test/embed"
test_dataset_files = sorted([f for f in os.listdir(test_path) if os.path.isfile(os.path.join(test_path, f)) and f[0]!="." and "tag" in f])

In [43]:
for test in test_dataset_files:
    with open(os.path.join(test_path, test), "rb") as f:
        data = pickle.load(f)
    X_embed = tags_pipeline.transform(data)
    with open(os.path.join(embed_path, test.replace('.pickle','_embed.pickle')), "wb") as f:
        pickle.dump(X_embed, f)

# Classification

## Load training data

In [49]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import xgboost as xgb
import multiprocessing
import numpy as np

In [50]:
tfidf_real_path = "./output/train/embed/webtext.train.human_embed.pickle"
tfidf_fake_path = "./output/train/embed/xl-1542M.train.machine_embed.pickle"
tag_real_path = "./output/train/embed/webtext.train.tag.human_embed.pickle"
tag_fake_path = "./output/train/embed/xl-1542M.train.tag.machine_embed.pickle"

In [51]:
with open(tfidf_real_path, "rb") as f:
    X_tfidf_real = pickle.load(f)
with open(tfidf_fake_path, "rb") as f:
    X_tfidf_fake = pickle.load(f)
X_tfidf = np.concatenate([X_tfidf_real, X_tfidf_fake])
y_train = [0 for _ in range(len(X_tfidf_real))] + [1 for _ in range(len(X_tfidf_fake))]

with open(tag_real_path, "rb") as f:
    X_tag_real = pickle.load(f)
with open(tag_fake_path, "rb") as f:
    X_tag_fake = pickle.load(f)
    
X_tag = np.concatenate([X_tag_real, X_tag_fake])
    
X_tfidf_tag = np.concatenate([X_tfidf, X_tag], axis=1)

## Train models

In [57]:
clf_tfidf = GridSearchCV(
        xgb.XGBClassifier(n_jobs=multiprocessing.cpu_count() // 2, tree_method="hist"),
        {"max_depth": [3, 4, 5], "n_estimators": [500, 1000, 1500]},
        verbose=1,
        n_jobs=2,
    )

clf_tag = GridSearchCV(
        xgb.XGBClassifier(n_jobs=multiprocessing.cpu_count() // 2, tree_method="hist"),
        {"max_depth": [3, 4, 5], "n_estimators": [500, 1000, 1500]},
        verbose=1,
        n_jobs=2,
    )

clf_tfidf_tag = GridSearchCV(
        xgb.XGBClassifier(n_jobs=multiprocessing.cpu_count() // 2, tree_method="hist"),
        {"max_depth": [3, 4, 5], "n_estimators": [500, 1000, 1500]},
        verbose=1,
        n_jobs=2,
    )

In [ ]:
%%time
clf_tfidf.fit(X_tfidf, y_train)

In [ ]:
pd.DataFrame(clf_tfidf.cv_results_)

In [ ]:
%%time
clf_tag.fit(X_tag, y_train)

In [ ]:
pd.DataFrame(clf_tag.cv_results_)

In [ ]:
%%time
clf_tfidf_tag.fit(X_tfidf_tag, y_train)

In [ ]:
pd.DataFrame(clf_tfidf_tag.cv_results_)

## Test models

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
embed_path = "./output/test/embed"
test_dataset_tag_files = sorted([f for f in os.listdir(embed_path) if os.path.isfile(os.path.join(embed_path, f)) and f[0]!="." and "tag" in f])

In [ ]:
test_dataset_files = [x.replace(".tag","") for x in test_dataset_tag_files]

In [ ]:
for test_dataset_tag_file in test_dataset_tag_files:
    test_dataset_file = test_dataset_tag_file.replace(".tag","")
    with open(os.path.join(embed_path, test_dataset_file)) as f:
        X_test_tfidf = pickle.load(f)
        label = int("machine" in test_dataset_file)
        y_test = [label for _ in range(len(X_test_tfidf))]
    with open(os.path.join(embed_path, test_dataset_tag_file)) as f:
        X_test_tag = pickle.load(f)
    X_test_tfidf_tag = np.concatenate([X_test_tfidf, X_test_tag], axis=1)
    y_pred = [round(yy) for yy in clf_tfidf.best_estimator_.predict(X_test_tfidf)]
    print(f'******\nTest dataset: {test_dataset_file.replace("_embed.pickle","")}')
    print('TF-IDF words:')
    print(accuracy_score(y_test, y_pred))
    print('TF-IDF tags:')
    y_pred = [round(yy) for yy in clf_tag.best_estimator_.predict(X_test_tag)]
    print(accuracy_score(y_test, y_pred))
    print('TF-IDF words+tags:')
    y_pred = [round(yy) for yy in clf_tfidf_tag.best_estimator_.predict(X_test_tfidf_tag)]
    print(accuracy_score(y_test, y_pred))